# Подготовка данных

Мы использользуем набор данных PadChest 

TODO: подготовить больше текста/описания для слайдов, см. https://bimcv.cipf.es/bimcv-projects/padchest/

In [76]:
!mkdir -p /content/padchest

# download sample images from padchest
!gdown 1FJ8T2lYCtTxtfhatpcXBpXvEBx9gTf81
!unzip -qo /content/sample.zip -d /content/padchest/
# download sample 2 images
!gdown 1_mKm1hlqAbxDfvZRKAJjW4icZPqAXICD
!unzip -qo /content/padchest_sample2.zip -d /content/padchest/
!ls /content/padchest/*.png | wc -l

# labels from ML drive
!gdown 1WYuYC-p77joqAv7TDivS3zGMD3NqTk4T
!mv /content/PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv /content/padchest/

Downloading...
From: https://drive.google.com/uc?id=1FJ8T2lYCtTxtfhatpcXBpXvEBx9gTf81
To: /content/sample.zip
100% 175M/175M [00:00<00:00, 217MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_mKm1hlqAbxDfvZRKAJjW4icZPqAXICD
To: /content/padchest_sample2.zip
100% 1.20G/1.20G [00:05<00:00, 218MB/s]
161
Downloading...
From: https://drive.google.com/uc?id=1WYuYC-p77joqAv7TDivS3zGMD3NqTk4T
To: /content/PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv
100% 99.2M/99.2M [00:01<00:00, 64.8MB/s]


Install pytorch X-ray library

In [77]:
!pip install torchxrayvision

In [78]:
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import torch  
import torchvision
import torchxrayvision as xrv
import pylab
import torch.optim as optim
import torch.nn as nn
from pathlib import Path
from tqdm import tqdm

In [79]:
PATH = Path('/content/padchest')

In [80]:
transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])
dataset = xrv.datasets.PC_Dataset(imgpath=PATH,csvpath=PATH/'PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv', transform=transform)

In [81]:
# consider records with existing png only
pngs = [f.name for f in PATH.glob("*.png")]
idxs = np.where(dataset.csv['ImageID'].isin(pngs))[0]
dataset = xrv.datasets.SubsetDataset(dataset, idxs)

In [82]:
print(dataset)

{'Air Trapping': {0.0: 131, 1.0: 8},
 'Aortic Atheromatosis': {0.0: 125, 1.0: 14},
 'Aortic Elongation': {0.0: 96, 1.0: 43},
 'Atelectasis': {0.0: 135, 1.0: 4},
 'Bronchiectasis': {0.0: 135, 1.0: 4},
 'Cardiomegaly': {0.0: 7, 1.0: 132},
 'Consolidation': {0.0: 139},
 'Costophrenic Angle Blunting': {0.0: 128, 1.0: 11},
 'Edema': {0.0: 139},
 'Effusion': {0.0: 136, 1.0: 3},
 'Emphysema': {0.0: 139},
 'Fibrosis': {0.0: 139},
 'Flattened Diaphragm': {0.0: 138, 1.0: 1},
 'Fracture': {0.0: 137, 1.0: 2},
 'Granuloma': {0.0: 136, 1.0: 3},
 'Hemidiaphragm Elevation': {0.0: 138, 1.0: 1},
 'Hernia': {0.0: 136, 1.0: 3},
 'Hilar Enlargement': {0.0: 118, 1.0: 21},
 'Infiltration': {0.0: 127, 1.0: 12},
 'Mass': {0.0: 138, 1.0: 1},
 'Nodule': {0.0: 134, 1.0: 5},
 'Pleural_Thickening': {0.0: 133, 1.0: 6},
 'Pneumonia': {0.0: 136, 1.0: 3},
 'Pneumothorax': {0.0: 138, 1.0: 1},
 'Scoliosis': {0.0: 122, 1.0: 17},
 'Support Devices': {0.0: 120, 1.0: 19},
 'Tube': {0.0: 139},
 'Tuberculosis': {0.0: 139}}
Sub

In [83]:
len_dataset=len(dataset)
n_train=int(0.9*len_dataset)
n_test=len_dataset-n_train
print(f'Total samples: {len_dataset}, train size size: {n_train}, test set size: {n_test}')

Total samples: 139, train size size: 125, test set size: 14


In [84]:
train_ds, test_ds = torch.utils.data.random_split(dataset, [n_train,n_test])
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=4,shuffle=True, num_workers=4)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=4,shuffle=True, num_workers=1)

In [85]:
model = xrv.models.DenseNet(num_classes=2).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Features and target columns

In [86]:
features = 'PA' # 
target = 'lab'  #

Training loop

In [89]:
train_loss_history, test_loss_history = [], []
for epoch in tqdm(range(10)):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        inputs=data[features].cuda()
        labels=data[target].long().cuda()
        labels=labels[:,2]
       
        # get the inputs; data is a list of [inputs, labels]
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    test_loss=0.0

    for i, data in enumerate(test_dl, 0):
        inputs=data[features].cuda()
        labels=data[target].long().cuda()
        labels=labels[:,2]

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # print statistics
        test_loss += loss.item()
        
    train_loss_history.append(running_loss)
    test_loss_history.append(test_loss)

0 0


KeyError: ignored

In [ ]:
plt.plot(train_loss_history, label='trainig loss')
plt.plot(test_loss_history, label='testing loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

to be continued.